In [32]:
# NOTE : only 20 most similar projects will be added
import pandas as pd
dataset_file_name = "tech_collab_hub_dataset.xlsx"
data = pd.read_excel(dataset_file_name)
data.head()

,repo_name,repo_author,repo_link,languages,topic,no_of_contributors,description,contributors,project_icon,proj_id
0,unspent,2qx,https://github.com/2qx/unspent,"CSS, TypeScript, Svelte, JavaScript, SCSS, HTML","automata, introspection, annuity, faucet, bch,...",0.0,"₿∙ϕ - Unspent Phi (an app, library, cli & db) ...",NaN,NaN,1
1,sqlite_gui,OceanDataTools,https://github.com/OceanDataTools/sqlite_gui,"CSS, JavaScript, Python, Shell, HTML",NaN,0.0,SQLite back-end for OceanDataTools/openrvdas. ...,NaN,NaN,2
2,Web-Components,notjawad404,https://github.com/notjawad404/Web-Components,"JavaScript, HTML, CSS",NaN,0.0,This project aims to create a website that co...,NaN,NaN,3
3,meza,freephile,https://github.com/freephile/meza,"JavaScript, Jinja, PHP, Shell, Python, CSS, Do...",NaN,0.0,Setup an enterprise MediaWiki server with simp...,NaN,NaN,4
4,notion-snap-reborn,Pietrucci-Blacher,https://github.com/Pietrucci-Blacher/notion-sn...,JavaScript,NaN,0.0,Remake of Notion-snap. Contribute to Pietrucci...,NaN,NaN,5


In [33]:
# after removing null value
# Drop rows where the 'repo_link' column is null
data = data.dropna(subset=['description'])
data = data.dropna(subset=['languages'])
data.shape

(1027, 10)

In [34]:
def clean_and_join_languages(languages):
    # print(languages)
    cleaned_languages = languages.split(', ')
    cleaned_languages_no_whitespace = [lang.replace(' ', '') for lang in cleaned_languages]
    return ' '.join(cleaned_languages_no_whitespace)
data["languages"] = data["languages"].apply(clean_and_join_languages)

In [35]:
# merge three column
if not data['topic'].isnull().any() and ' ' not in data['topic'].values:
    data['tags'] = data['languages'] + ". " + data['topic'] + '. ' + data['description']
else:
    data['tags'] = data['languages'] + '. ' + data['description']

In [36]:
# make all the char in tag to lowercase
data['tags'] = data['tags'].apply(lambda x: x.lower())

In [37]:
from nltk.stem import PorterStemmer

# Initialize PorterStemmer
porter = PorterStemmer()

# Define a function to perform stemming on each word
def stem_words(text):
    return ' '.join([porter.stem(word) for word in text.split()])

# Apply stemming to the 'tags' column
data['tags'] = data['tags'].apply(stem_words)

In [38]:
# printing first row after processing tag
data.iloc[0]['tags']

'css typescript svelt javascript scss html. ₿∙ϕ - unspent phi (an app, library, cli & db) for execut a set of anyone-can-spend contract on bitcoin cash. - 2qx/unspent'

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=3, max_features = None, strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3), stop_words='english')

In [40]:
# Fit and transform the 'tags' column to TF-IDF vectors
tfidf_matrix = tfidf_vectorizer.fit_transform(data['tags'])

In [41]:
from sklearn.metrics.pairwise import cosine_similarity
# Calculate cosine similarity matrix
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [43]:
import numpy as np
print(similarity_matrix[0])
# # Display the similarity matrix
first_row_similarity = similarity_matrix[0]

# Get the indices of sorted similarity scores in descending order
sorted_indices = np.argsort(first_row_similarity)[::-1]
print(sorted_indices)

# displaying most similar top 5 projects
for index in range(0, 15):
    row = sorted_indices[index]
    print(str(index+1), end = " : ")
    print('id :',data.iloc[row]['proj_id'],' link=',data.iloc[row]['repo_link'])
    
    print()

[1.         0.02016249 0.01248489 ... 0.15518822 0.00447497 0.00767282]
[  0 492 596 ... 365 371 335]
1 : id 1  link= https://github.com/2qx/unspent

2 : id 494  link= https://github.com/Moha0170/kaloria

3 : id 598  link= https://github.com/SauceLLC/sauce4strava

4 : id 634  link= https://github.com/majidh1/JalaliDatePicker

5 : id 753  link= https://github.com/code4romania/covid-19-cine-ce-face

6 : id 242  link= https://github.com/abbasmurudkar/CHAT-APPLICATION

7 : id 1007  link= https://github.com/strapi/community-content

8 : id 801  link= https://github.com/3DStreet/3dstreet-editor

9 : id 536  link= https://github.com/adridadou/eth-contract-api

10 : id 512  link= https://github.com/JenkinsMD/GrpPrj1-Bitcoin

11 : id 629  link= https://github.com/open-ibc/ibc-app-solidity-template

12 : id 732  link= https://github.com/GSA/GEAR3

13 : id 254  link= https://github.com/RUDRANSHFLY/mothers_love

14 : id 669  link= https://github.com/GemeenteNijmegen/mijn-nijmegen

15 : id 300  lin

In [65]:
import numpy as np
import pandas as pd
df = pd.read_excel('tech_collab_hub_dataset.xlsx')

for row in similarity_matrix:
    sorted_indices = np.argsort(row)[::-1]
    similarity_array = [] #store similarity by proj_id
    print(sorted_indices)

    for index, column in enumerate(sorted_indices):
        if index != 0:
            similarity_array.append(data.iloc[column]['proj_id'])
        if index > 20:
            break
    print(similarity_array)
    if similarity_array:
        proj_index_in_excel = data.iloc[sorted_indices[0]]['proj_id'] - 1
        similarity_array_string = ', '.join([str(item) for item in similarity_array])
        df.loc[proj_index_in_excel, 'similarity_matrix'] = similarity_array_string
        print('done ', proj_index_in_excel)
df.to_excel('tech_collab_hub_dataset.xlsx', index=False)

[  0 492 596 ... 365 371 335]
[494, 598, 634, 753, 242, 1007, 801, 536, 512, 629, 732, 254, 669, 300, 976, 1027, 943, 487, 93, 66, 312]
done  0
[  1 446 920 ... 350 840 830]
[448, 923, 886, 567, 287, 57, 387, 5, 722, 959, 18, 546, 227, 828, 736, 687, 563, 88, 871, 569, 14]
done  1
[  2 571 753 ... 855  29 929]
[573, 756, 409, 81, 220, 583, 117, 889, 761, 174, 207, 968, 177, 128, 381, 660, 717, 319, 870, 994, 623]
done  2
[  3 251 909 ... 131 130 301]
[252, 912, 27, 946, 406, 403, 436, 872, 563, 135, 20, 771, 434, 69, 955, 603, 63, 284, 100, 679, 822]
done  3
[719 385   4 ...  99 669 330]
[387, 5, 57, 287, 546, 147, 246, 143, 19, 192, 317, 288, 449, 99, 581, 386, 227, 41, 215, 212, 404]
done  721
[  5 428 517 ... 469 460 534]
[430, 519, 457, 716, 310, 760, 657, 655, 272, 90, 613, 142, 397, 278, 601, 169, 181, 71, 664, 546, 977]
done  5
[   6 1024  212 ...  744  946  869]
[1027, 213, 595, 660, 838, 955, 516, 947, 496, 300, 847, 220, 205, 494, 434, 287, 57, 387, 722, 5, 326]
done  6
[  7 